In [1]:
import timeit
start=timeit.default_timer()

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

## Load files

In [8]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [9]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    return text

In [10]:
def clean_text(text):
    text=text.lower()
    text=text.replace("ï»¿","")
    text=text.replace("\n"," ")
    text=text.replace("*"," ")
    text=text.replace("-"," ")
    text=text.replace("\'s"," is")
    text=text.replace("n\'t"," not")
    text=text.replace("\'m"," am")
    text=text.replace("\'ll"," will")
    text=text.replace("in'","ing")
    text=text.replace("mr.","mr")
    text=text.replace("mrs.","mrs")
    text=text.replace("i'd","i would")
    text=text.replace("p. s.","")
    text=text.replace("n. y.","")
    text=text.replace("dec.","dec")
    text=text.replace("1904.","1904")
    text=text.replace("austin.","austin")
    text=text.replace("co.","co")
    text=text.replace("new york.","new york")
    text=text.replace("kringle.","kringle")
    text=text.replace("dear.","dear ")
    text=text.replace("not.","not ")
    text=text.replace("do.","do ")
    text=text.replace("nan.","nan ")
    text=text.replace("yes.","yes ")
    text=text.replace("song.","song ")
    text=text.replace("boy.","boy ")
    text=text.replace("will.","will ")
    text=text.replace("now.","now ")
    text=text.replace("child.","child ")
    text=text.replace("reconciled.","reconciled ")
    
    text=text.replace('"'," ")
    text=text.replace('_'," ")
    text=text.replace('?'," ")
    text=text.replace('!'," ")
    text=text.replace(','," ")
    text=text.replace(":"," ")
    text=text.replace(";"," ")
    text=text.replace("...","")
       
    text=re.sub("[\(\[].*?[\)\]]"," ",text)
 
    return text

In [11]:
text=load_text(filename[9])

In [12]:
text=clean_text(text)
text

"chapter i  the coming of joan   along some parts of the coast in south wales the mountains rise abruptly from the shore  with only a narrow shingle between them and the sea.  high above the coast  however  there are warm  sunny little valleys and dells among the hills  where sheep can find pasture and a fold  and here there are many small farmsteads  surrounded by wild rocks and bleak uplands  where the farmer and his family live with their servants  if they happen to have any  as they used to do in old times  sitting in the same kitchen  and taking their meals together as one household.  miss priscilla parry was the last of three leaseholders of one of these little farms. her grandfather had enclosed the meadows and the corn fields from the open mountain  on condition that he should have a lease for three lives from the owner of the land. his own and his son is had been two of the lives  and priscilla is was the third.  the farm was poor  for the land was hard to cultivate. in every 

In [13]:
text=' '.join(text.split())
text

"chapter i the coming of joan along some parts of the coast in south wales the mountains rise abruptly from the shore with only a narrow shingle between them and the sea. high above the coast however there are warm sunny little valleys and dells among the hills where sheep can find pasture and a fold and here there are many small farmsteads surrounded by wild rocks and bleak uplands where the farmer and his family live with their servants if they happen to have any as they used to do in old times sitting in the same kitchen and taking their meals together as one household. miss priscilla parry was the last of three leaseholders of one of these little farms. her grandfather had enclosed the meadows and the corn fields from the open mountain on condition that he should have a lease for three lives from the owner of the land. his own and his son is had been two of the lives and priscilla is was the third. the farm was poor for the land was hard to cultivate. in every field there were plac

In [14]:
temp_list=text.split(".")
temp_list

['chapter i the coming of joan along some parts of the coast in south wales the mountains rise abruptly from the shore with only a narrow shingle between them and the sea',
 ' high above the coast however there are warm sunny little valleys and dells among the hills where sheep can find pasture and a fold and here there are many small farmsteads surrounded by wild rocks and bleak uplands where the farmer and his family live with their servants if they happen to have any as they used to do in old times sitting in the same kitchen and taking their meals together as one household',
 ' miss priscilla parry was the last of three leaseholders of one of these little farms',
 ' her grandfather had enclosed the meadows and the corn fields from the open mountain on condition that he should have a lease for three lives from the owner of the land',
 ' his own and his son is had been two of the lives and priscilla is was the third',
 ' the farm was poor for the land was hard to cultivate',
 ' in ev

In [15]:
x

NameError: name 'x' is not defined

In [ ]:
def clean_text(text):
    text=text.lower()
    text=text.replace("ï»¿","")
    text=text.replace(","," ")
    text=text.replace("\n"," ")
    text=text.replace("*"," ")
    text=text.replace(":"," ")
    text=text.replace(":"," ")
    text=re.sub("[\(\[].*?[\)\]]"," ",text)
    text = re.sub(r'--',' ',text)
    text= text.replace("http //www.gutenberg.net"," ")
    text=text.replace("www.gutenberg.net"," ")
    text=text.replace("14606-h.htm"," ")

    text=text.replace("i."," ")
    text=text.replace("ii."," ")
    text=text.replace("iii."," ")
    text=text.replace("iv."," ")
    text=text.replace("v."," ")
    text=text.replace("vi."," ")
    text=text.replace("vi."," ")
    text=text.replace("vii."," ")
    text=text.replace("viii."," ")
    text=text.replace("ix."," ")
    text=text.replace("x."," ")
    text=text.replace("xi."," ")

    text=text.replace('\'',"")
    text=text.replace('/',"")
    text=text.replace('14606-h.z',"")
    text=text.replace('iso-646-us',"")
    text=text.replace('?'," ")
    text = re.sub('\S*@\S*\s?',' ',text)
    text=text.replace('"'," ")

    return text